In [1]:
import os
import glob
import time
import math
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import soundfile as sf
import librosa
import librosa.display

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             accuracy_score, roc_curve, auc, confusion_matrix,
                             average_precision_score, precision_recall_curve)
from scipy.stats import gaussian_kde
from tqdm import tqdm

In [2]:
try:
    from IPython.display import Audio as IPyAudio, display as ipy_display
except Exception:
    IPyAudio = None
    ipy_display = None

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

Device: cuda


In [3]:
class Config:
    SAMPLE_RATE = 16000
    N_FFT = 512
    HOP_LENGTH = 256
    N_MELS = 128
    AUDIO_DURATION = 3.0            
    LATENT_DIM = 64
    HIDDEN_DIMS = [128, 256, 512, 1024]
    BETA = 1.0
    BATCH_SIZE = 16
    EPOCHS = 200                    
    LEARNING_RATE = 1e-3
    NUM_SAMPLES = 1000              
    CONTAMINATION = 0.1
    DATA_PATH = './librispeech_data' 
    OUTPUT_PATH = './output'
    VIS_SAMPLES = 5
    TSNE_MAX = 1000

config = Config()

os.makedirs(config.OUTPUT_PATH, exist_ok=True)
os.makedirs(os.path.join(config.OUTPUT_PATH, "audio"), exist_ok=True)
os.makedirs(os.path.join(config.OUTPUT_PATH, "plots"), exist_ok=True)
os.makedirs(os.path.join(config.OUTPUT_PATH, "models"), exist_ok=True)

BEST_MODEL_PATH = os.path.join(config.OUTPUT_PATH, "models", "best_vae_model.pth")

In [4]:
class LibriSpeechVAEDataset(Dataset):

    def __init__(self, root_dir, sample_rate=16000, n_mels=128, n_fft=512, hop_length=256, duration=3.0, num_samples=None):
        self.root_dir = os.path.abspath(root_dir)
        self.sample_rate = sample_rate
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.duration = duration
        self.target_length = int(sample_rate * duration)

        patterns = ['**/*.wav', '**/*.flac', '**/*.mp3', '**/*.m4a', '**/*.aac']
        files = []
        for p in patterns:
            files.extend(glob.glob(os.path.join(self.root_dir, p), recursive=True))
        files = sorted(files)
        if num_samples:
            files = files[:min(num_samples, len(files))]
        if len(files) == 0:
            raise RuntimeError(f"No audio files found under {self.root_dir}. Place audio (.wav/.flac) files there.")
        self.files = files
        print(f"Found {len(self.files)} audio files under {self.root_dir}")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path = self.files[idx]
        # load with librosa (resamples automatically)
        y, sr = librosa.load(path, sr=self.sample_rate, mono=True)
        # trim/pad to duration
        if len(y) > self.target_length:
            start = np.random.randint(0, len(y) - self.target_length + 1)
            y = y[start:start + self.target_length]
        elif len(y) < self.target_length:
            pad = self.target_length - len(y)
            y = np.pad(y, (0, pad), mode='constant')

        # mel spectrogram (power)
        mel_power = librosa.feature.melspectrogram(y=y, sr=self.sample_rate,
                                                   n_fft=self.n_fft, hop_length=self.hop_length,
                                                   n_mels=self.n_mels, power=2.0)
        mel_log = np.log(mel_power + 1e-9)
        # normalize per-sample (zero mean, unit std)
        mel_norm = (mel_log - mel_log.mean()) / (mel_log.std() + 1e-9)

        mel_norm_t = torch.from_numpy(mel_norm).float()      # [n_mels, time]
        return mel_norm_t, y.astype(np.float32), mel_power.astype(np.float32), mel_log.astype(np.float32)

In [5]:
class Encoder(nn.Module):
    def __init__(self, in_channels, hidden_dims, latent_dim):
        super().__init__()
        modules = []
        c = in_channels
        for h in hidden_dims:
            modules.append(nn.Sequential(
                nn.Conv2d(c, h, kernel_size=3, stride=2, padding=1),
                nn.BatchNorm2d(h),
                nn.LeakyReLU(0.2),
                nn.Dropout2d(0.1)
            ))
            c = h
        self.net = nn.Sequential(*modules)
        self.flatten_size = None
        self.fc_mu = None
        self.fc_logvar = None
        self.latent_dim = latent_dim

    def forward(self, x):
        x = self.net(x)
        if self.flatten_size is None:
            self.flatten_size = x.shape[1] * x.shape[2] * x.shape[3]
            self.fc_mu = nn.Linear(self.flatten_size, self.latent_dim).to(x.device)
            self.fc_logvar = nn.Linear(self.flatten_size, self.latent_dim).to(x.device)
        x = x.view(x.size(0), -1)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar, self.flatten_size

In [6]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dims, out_channels):
        super().__init__()
        self.latent_dim = latent_dim
        self.hidden_dims = hidden_dims
        self.fc = None
        modules = []
        rev = hidden_dims[::-1]
        for i in range(len(rev)-1):
            modules.append(nn.Sequential(
                nn.ConvTranspose2d(rev[i], rev[i+1], kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.BatchNorm2d(rev[i+1]),
                nn.LeakyReLU(0.2),
                nn.Dropout2d(0.1)
            ))
        modules.append(nn.Sequential(
            nn.ConvTranspose2d(rev[-1], out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        ))
        self.net = nn.Sequential(*modules)

    def forward(self, z, flatten_size, spatial_size):
        if self.fc is None:
            self.fc = nn.Linear(self.latent_dim, flatten_size).to(z.device)
        x = self.fc(z)
        channels = self.hidden_dims[-1]
        x = x.view(x.size(0), channels, spatial_size[0], spatial_size[1])
        x = self.net(x)
        return x

In [7]:
class SpeechVAE(nn.Module):
    def __init__(self, input_channels=1, hidden_dims=None, latent_dim=64, beta=1.0):
        super().__init__()
        if hidden_dims is None:
            hidden_dims = [128, 256, 512, 1024]
        self.encoder = Encoder(input_channels, hidden_dims, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dims, input_channels)
        self.beta = beta
        self.spatial_size = None
        self.latent_dim = latent_dim

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        mu, logvar, flatten_size = self.encoder(x)
        if self.spatial_size is None:
            with torch.no_grad():
                tmp = self.encoder.net(x)
                self.spatial_size = (tmp.shape[2], tmp.shape[3])
        z = self.reparameterize(mu, logvar)
        recon = self.decoder(z, flatten_size, self.spatial_size)
        if recon.shape != x.shape:
            recon = F.interpolate(recon, size=(x.shape[2], x.shape[3]), mode='bilinear', align_corners=False)
        return recon, mu, logvar, z

    def loss_function(self, recon, x, mu, logvar):
        recon_loss = F.mse_loss(recon, x, reduction='sum')
        kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return recon_loss + self.beta * kld, recon_loss, kld

    def get_reconstruction_error(self, x):
        self.eval()
        with torch.no_grad():
            recon, mu, logvar, z = self(x)
            e = F.mse_loss(recon, x, reduction='none')
            e = e.view(e.size(0), -1).mean(dim=1)
            return e

In [8]:
class MetricsCalculator:
    @staticmethod
    def mse(a,b): return np.mean((a-b)**2)
    @staticmethod
    def mae(a,b): return np.mean(np.abs(a-b))
    @staticmethod
    def psnr(a,b, max_val=1.0):
        mse = np.mean((a-b)**2)
        if mse == 0: return 100.0
        return 20 * np.log10(max_val / math.sqrt(mse))
    @staticmethod
    def snr(a,b):
        sp = np.mean(a**2)
        npow = np.mean((a-b)**2)
        if npow == 0: return 100.0
        return 10 * np.log10(sp / npow)
    @staticmethod
    def ssim(a,b):
        c1 = 0.01**2; c2 = 0.03**2
        mu1, mu2 = a.mean(), b.mean()
        s1, s2 = a.var(), b.var()
        cov = np.mean((a-mu1)*(b-mu2))
        return ((2*mu1*mu2 + c1)*(2*cov + c2))/((mu1**2 + mu2**2 + c1)*(s1+s2+c2)+1e-12)
    @staticmethod
    def spectral_convergence(a,b):
        num = np.linalg.norm(a-b, ord='fro')
        den = np.linalg.norm(a, ord='fro')
        return num/(den+1e-9)

In [9]:
class AudioReconstructor:
    def __init__(self, sr, n_fft, hop_length, n_mels):
        self.sr = sr
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels

    def reconstruct_audio(self, recon_norm_log, reference_mel_log):
        """
        recon_norm_log: numpy [n_mels, t] => model output normalized log-mel
        reference_mel_log: numpy [n_mels, t] => original mel_log for denorm stats
        """
        try:
            # Denormalize approx using reference mel_log mean/std
            ref_mean = reference_mel_log.mean()
            ref_std = reference_mel_log.std()
            recon_log = recon_norm_log * (ref_std + 1e-9) + ref_mean
            recon_power = np.exp(recon_log)  # mel power
            # invert mel to audio using librosa (Griffin-Lim inside)
            y = librosa.feature.inverse.mel_to_audio(M=recon_power, sr=self.sr, n_fft=self.n_fft,
                                                     hop_length=self.hop_length, power=2.0, n_iter=64)
            # normalize audio to -0.99..0.99
            maxv = np.max(np.abs(y)) + 1e-9
            y = y / maxv * 0.99
            return y
        except Exception as e:
            # fallback None
            return None

In [10]:
def train_vae(model, train_loader, val_loader, epochs, lr, device):
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=8, factor=0.5)
    train_losses, val_losses, recon_losses, kl_losses = [], [], [], []
    best_val = float('inf'); patience = 12; pcount = 0

    for epoch in range(epochs):
        model.train()
        tloss = trecon = tkl = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for mel_norm, _, _, _ in pbar:
            x = mel_norm.unsqueeze(1).to(device)   # [B,1,H,W]
            optimizer.zero_grad()
            recon, mu, logvar, _ = model(x)
            loss, recon_l, kld = model.loss_function(recon, x, mu, logvar)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            tloss += loss.item()
            trecon += recon_l.item()
            tkl += kld.item()
            pbar.set_postfix({'loss': (loss.item()/x.size(0))})
        avg_train = tloss / len(train_loader.dataset)
        avg_recon = trecon / len(train_loader.dataset)
        avg_kl = tkl / len(train_loader.dataset)
        train_losses.append(avg_train); recon_losses.append(avg_recon); kl_losses.append(avg_kl)

        model.eval()
        vloss = 0.0
        with torch.no_grad():
            for mel_norm, _, _, _ in val_loader:
                x = mel_norm.unsqueeze(1).to(device)
                recon, mu, logvar, _ = model(x)
                loss, _, _ = model.loss_function(recon, x, mu, logvar)
                vloss += loss.item()
        avg_val = vloss / len(val_loader.dataset)
        val_losses.append(avg_val)

        scheduler.step(avg_val)

        if avg_val < best_val:
            best_val = avg_val
            torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'val_loss': avg_val}, BEST_MODEL_PATH)
            pcount = 0
        else:
            pcount += 1

        print(f"Epoch {epoch+1}/{epochs} | Train: {avg_train:.4f} | Val: {avg_val:.4f} | Recon: {avg_recon:.4f} | KL: {avg_kl:.4f}")

        if pcount >= patience:
            print("Early stopping triggered.")
            break

    return train_losses, val_losses, recon_losses, kl_losses

In [11]:
def evaluate_model(model, test_loader, device, audio_reconstructor):
    model.eval()
    metrics = {'mse':[], 'mae':[], 'psnr':[], 'snr':[], 'ssim':[], 'spectral_conv':[]}
    orig_specs, recon_specs, latents = [], [], []
    orig_audio, recon_audio = [], []

    with torch.no_grad():
        for batch_idx, (mel_norm, waveform, mel_power, mel_log) in enumerate(tqdm(test_loader, desc="Evaluating")):
            if batch_idx >= 8:  # limit for memory/time
                break
            x = mel_norm.unsqueeze(1).to(device)
            recon, mu, logvar, _ = model(x)
            recon_np = recon.cpu().numpy()
            orig_np = x.cpu().numpy()
            for i in range(recon_np.shape[0]):
                orig_spec = orig_np[i,0]
                recon_spec = recon_np[i,0]
                metrics['mse'].append(MetricsCalculator.mse(orig_spec, recon_spec))
                metrics['mae'].append(MetricsCalculator.mae(orig_spec, recon_spec))
                metrics['psnr'].append(MetricsCalculator.psnr(orig_spec, recon_spec))
                metrics['snr'].append(MetricsCalculator.snr(orig_spec, recon_spec))
                metrics['ssim'].append(MetricsCalculator.ssim(orig_spec, recon_spec))
                metrics['spectral_conv'].append(MetricsCalculator.spectral_convergence(orig_spec, recon_spec))

                # reconstruct audio for first few samples
                if len(orig_audio) < config.VIS_SAMPLES:
                    orig_audio.append(waveform[i].numpy() if isinstance(waveform[i], np.ndarray) else waveform[i].numpy())
                    # recon_spec is normalized log-mel — use corresponding mel_log for denorm stats
                    ref_log = mel_log[i].numpy() if isinstance(mel_log, torch.Tensor) else mel_log[i]
                    recon_audio_sample = audio_reconstructor.reconstruct_audio(recon_spec, ref_log)
                    recon_audio.append(recon_audio_sample)

            orig_specs.append(torch.from_numpy(orig_np))
            recon_specs.append(torch.from_numpy(recon_np))
            latents.append(mu.cpu())

    avg = {k: np.mean(v) if len(v)>0 else float('nan') for k,v in metrics.items()}
    std = {k: np.std(v) if len(v)>0 else float('nan') for k,v in metrics.items()}
    return avg, std, orig_specs, recon_specs, latents, orig_audio, recon_audio, metrics

In [12]:
def evaluate_anomaly_detection(model, test_loader, device):
    # collect small pool
    pool = []
    for i, (mel_norm, _, _, _) in enumerate(test_loader):
        pool.append(mel_norm)
        if i >= 80: break
    if len(pool) == 0:
        raise RuntimeError("Not enough data for anomaly evaluation.")
    pool = torch.cat(pool, dim=0)  # [N, H, W]

    # normal and anomalies (add noise)
    N = min(200, pool.size(0))
    normal = pool[:N]
    anomaly = normal + torch.randn_like(normal) * 2.0

    num_anom = max(1, int(N * config.CONTAMINATION))
    test_data = torch.cat([normal, anomaly[:num_anom]], dim=0)
    test_labels = np.concatenate([np.zeros(N), np.ones(num_anom)])

    # compute reconstruction errors
    model.eval()
    errors = []
    with torch.no_grad():
        for i in range(0, len(test_data), 32):
            batch = test_data[i:i+32].unsqueeze(1).to(device)
            err = model.get_reconstruction_error(batch)
            errors.extend(err.cpu().numpy())
    errors = np.array(errors)
    threshold = np.percentile(errors, 95)
    preds = (errors > threshold).astype(int)

    acc = accuracy_score(test_labels, preds)
    prec = precision_score(test_labels, preds, zero_division=0)
    rec = recall_score(test_labels, preds, zero_division=0)
    f1 = f1_score(test_labels, preds, zero_division=0)
    fpr, tpr, _ = roc_curve(test_labels, errors)
    roc_auc = auc(fpr, tpr)
    avg_prec = average_precision_score(test_labels, errors)
    cm = confusion_matrix(test_labels, preds)

    return {
        'accuracy': acc, 'precision': prec, 'recall': rec, 'f1_score': f1,
        'roc_auc': roc_auc, 'avg_precision': avg_prec, 'confusion_matrix': cm,
        'fpr': fpr, 'tpr': tpr, 'errors': errors, 'predictions': preds,
        'labels': test_labels, 'threshold': threshold
    }

In [13]:
def save_audio(original_audio, reconstructed_audio, sr):
    for i in range(min(len(original_audio), config.VIS_SAMPLES)):
        sf.write(os.path.join(config.OUTPUT_PATH, "audio", f"original_{i+1}.wav"), original_audio[i], sr)
        if reconstructed_audio[i] is not None:
            sf.write(os.path.join(config.OUTPUT_PATH, "audio", f"reconstructed_{i+1}.wav"), reconstructed_audio[i], sr)
    print("Saved audio samples to", os.path.join(config.OUTPUT_PATH, "audio"))

def plot_training_curves(train_losses, val_losses, recon_losses, kl_losses):
    epochs = list(range(1, len(train_losses)+1))
    plt.figure(figsize=(10,6))
    plt.plot(epochs, train_losses, label='train')
    plt.plot(epochs, val_losses, label='val')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.title('Total Loss')
    plt.savefig(os.path.join(config.OUTPUT_PATH, "plots", "training_curves.png"), dpi=200)
    plt.close()

def plot_reconstruction_examples(original_specs, reconstructed_specs, n=4):
    if len(original_specs)==0: return
    n = min(n, len(original_specs[0]))
    fig, axes = plt.subplots(n, 3, figsize=(12, 3*n))
    for i in range(n):
        orig = original_specs[0][i,0].numpy()
        recon = reconstructed_specs[0][i,0].numpy()
        axes[i,0].imshow(orig, origin='lower', aspect='auto'); axes[i,0].set_title('Original')
        axes[i,1].imshow(recon, origin='lower', aspect='auto'); axes[i,1].set_title('Reconstructed')
        axes[i,2].imshow(np.abs(orig-recon), origin='lower', aspect='auto'); axes[i,2].set_title('Abs Diff')
    plt.tight_layout()
    plt.savefig(os.path.join(config.OUTPUT_PATH, "plots", "reconstruction_examples.png"), dpi=200)
    plt.close()

def plot_latent_tsne(latents):
    if len(latents)==0: return
    X = torch.cat(latents, dim=0).numpy()
    n = min(len(X), config.TSNE_MAX)
    Xs = X[:n]
    print("Computing t-SNE (this may take a while)...")
    tsne = TSNE(n_components=2, random_state=42, perplexity=30)
    Z = tsne.fit_transform(Xs)
    plt.figure(figsize=(8,6))
    plt.scatter(Z[:,0], Z[:,1], s=6, alpha=0.6)
    plt.title('Latent t-SNE')
    plt.savefig(os.path.join(config.OUTPUT_PATH, "plots", "latent_tsne.png"), dpi=200)
    plt.close()

In [14]:
def main():
    t0 = time.time()
    print("\n=== Speech VAE (librosa-based) ===\n")
    print("Data folder:", os.path.abspath(config.DATA_PATH))
    # build dataset
    dataset = LibriSpeechVAEDataset(config.DATA_PATH, sample_rate=config.SAMPLE_RATE,
                                    n_mels=config.N_MELS, n_fft=config.N_FFT,
                                    hop_length=config.HOP_LENGTH, duration=config.AUDIO_DURATION,
                                    num_samples=config.NUM_SAMPLES)
    N = len(dataset)
    train_n = int(0.7 * N); val_n = int(0.15 * N); test_n = N - train_n - val_n
    train_ds, val_ds, test_ds = torch.utils.data.random_split(dataset, [train_n, val_n, test_n],
                                                               generator=torch.Generator().manual_seed(42))
    train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_ds, batch_size=config.BATCH_SIZE, shuffle=False, num_workers=0)

    print(f"Dataset splits -> train: {train_n}, val: {val_n}, test: {test_n}")

    model = SpeechVAE(input_channels=1, hidden_dims=config.HIDDEN_DIMS, latent_dim=config.LATENT_DIM, beta=config.BETA).to(device)
    params = sum(p.numel() for p in model.parameters())
    print("Model parameters:", params)

    train_losses, val_losses, recon_losses, kl_losses = train_vae(model, train_loader, val_loader, config.EPOCHS, config.LEARNING_RATE, device)

    if os.path.exists(BEST_MODEL_PATH):
        ck = torch.load(BEST_MODEL_PATH, map_location=device)
        model.load_state_dict(ck['model_state_dict'] if 'model_state_dict' in ck else ck)
        print("Loaded best model from", BEST_MODEL_PATH)

    audio_reconstructor = AudioReconstructor(config.SAMPLE_RATE, config.N_FFT, config.HOP_LENGTH, config.N_MELS)
    avg_metrics, std_metrics, orig_specs, recon_specs, latents, orig_audio, recon_audio, all_metrics = evaluate_model(model, test_loader, device, audio_reconstructor)

    anomaly_metrics = evaluate_anomaly_detection(model, test_loader, device)

    if len(orig_audio) > 0:
        save_audio(orig_audio, recon_audio, config.SAMPLE_RATE)

    plot_training_curves(train_losses, val_losses, recon_losses, kl_losses)
    plot_reconstruction_examples(orig_specs, recon_specs, n=min(4, len(orig_specs[0]) if len(orig_specs)>0 else 0))
    plot_latent_tsne(latents)

    try:
        import matplotlib
        # ROC
        plt.figure(figsize=(6,6))
        plt.plot(anomaly_metrics['fpr'], anomaly_metrics['tpr'], label=f"AUC={anomaly_metrics['roc_auc']:.4f}")
        plt.plot([0,1],[0,1], linestyle='--', color='k')
        plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC'); plt.legend()
        plt.savefig(os.path.join(config.OUTPUT_PATH, "plots", "anomaly_roc.png"), dpi=200)
        plt.close()
    except Exception:
        pass

    print("\n--- Reconstruction metrics (averages) ---")
    for k,v in avg_metrics.items():
        print(f"{k}: {v:.4f} (std {std_metrics[k]:.4f})")
    print("\n--- Anomaly detection ---")
    for k in ['accuracy','precision','recall','f1_score','roc_auc','avg_precision']:
        print(f"{k}: {anomaly_metrics[k]:.4f}")

    if IPyAudio is not None and len(orig_audio)>0:
        print("\nPlaying original (first) and reconstructed (first) audio (notebook only):")
        ipy_display(IPyAudio(orig_audio[0], rate=config.SAMPLE_RATE))
        if recon_audio[0] is not None:
            ipy_display(IPyAudio(recon_audio[0], rate=config.SAMPLE_RATE))
        else:
            print("No reconstructed audio available for playback.")

    print("\nOutputs saved to:", os.path.abspath(config.OUTPUT_PATH))
    print("Total time (min):", (time.time()-t0)/60.0)

if __name__ == "__main__":
    main()



=== Speech VAE (librosa-based) ===

Data folder: C:\Users\koust\Koustab Projects\VAE_LVM\Speech\librispeech_data
Found 1000 audio files under C:\Users\koust\Koustab Projects\VAE_LVM\Speech\librispeech_data
Dataset splits -> train: 700, val: 150, test: 150
Model parameters: 12397057


Epoch 1/200: 100%|███████████████████████████████████████████████████████| 44/44 [00:05<00:00,  8.39it/s, loss=1.96e+4]


Epoch 1/200 | Train: 22291.3098 | Val: 19138.9977 | Recon: 22254.2127 | KL: 37.0970


Epoch 2/200: 100%|███████████████████████████████████████████████████████| 44/44 [00:03<00:00, 13.04it/s, loss=1.44e+4]


Epoch 2/200 | Train: 17324.8781 | Val: 15580.3573 | Recon: 17276.7410 | KL: 48.1373


...

Evaluating:  80%|████████████████████████████████████████████████████████▊              | 8/10 [00:01<00:00,  4.24it/s]


Saved audio samples to ./output\audio
Computing t-SNE (this may take a while)...


  File "C:\Users\koust\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\koust\anaconda3\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\koust\anaconda3\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                        pass_fds, cwd, env,
                        ^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
                        gid, gids, uid, umask,
                        ^^^^^^^^^^^^^^^^^^^^^^
                        start_new_session, process_group)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\koust\anaconda3\Lib\subprocess.


--- Reconstruction metrics (averages) ---
mse: 0.3187 (std 0.0766)
mae: 0.4427 (std 0.0561)
psnr: 5.1024 (std 1.1227)
snr: 5.1024 (std 1.1227)
ssim: 0.2672 (std 0.2427)
spectral_conv: 0.5603 (std 0.0695)

--- Anomaly detection ---
accuracy: 0.9636
precision: 1.0000
recall: 0.6000
f1_score: 0.7500
roc_auc: 1.0000
avg_precision: 1.0000

Playing original (first) and reconstructed (first) audio (notebook only):



Outputs saved to: C:\Users\koust\Koustab Projects\VAE_LVM\Speech\output
Total time (min): 9.142680438359578
